# Практическая работа по лекциям 10 и 11
## Поймай меня, если сможешь
## Преамбула
Будем решать `задачу идентификации злоумышленника` по его поведению в сети Интернет. Если коротко, то он будет вести себя совсем не так, как обычный человек. Если это взломщик почтовых ящиков, то, скорее всего, в отличие от владельца ящика, он будет ставить флажки на письмах по-другому, иначе перемещать курсор мышки, удалять или отправлять письма с нетипичной для владельца тематикой.

В нашем случае необходимо решить похожую задачу: алгоритм будет анализировать `последовательность из нескольких веб-сайтов, посещенных подряд одним и тем же человеком`, и определять, владелец это или злоумышленник (кто-то другой).


### Данные
- `practice_anomaly_data.csv` $-$ таблица с данными о сессиях пользователей. Данные были собраны с прокси-серверов Университета Блеза Паскаля. 
- `site.pkl` $-$ словарь с сайтами.

Изначально данные были размечены, но я удалил столбец с метками, чтобы можно было решить задачу поиска аномалий `методами обучения без учителя`. 

### Признаки пользователей
- Признаки, названия которых начинаются с `site`, это номера сайтов, которые посетил пользователь в конкретную сессию. По номеру сайта можно найти ссылку в файле `site.pkl`;
- Признаки, названия которых начинаются с `time`, являются временными метками, когда пользователь посетил сайт.
- Допустимо ожидаемый процент аномалий меньше 1% (примерно 0.9%)

## Ход работы
### 1. Загрузка данных
1. Загрузите данные из файла `practice_anomaly_data.csv`, используя атрибут `session_id` в качестве индекса.

In [ ]:
# ваш код здесь

2. Используя модуль `pickle`* языка программирования `python`, загрузите содержимое словаря с сайтами из файла `site.pkl`.

*Краткое описание модуля <a href=https://pythonworld.ru/moduli/modul-pickle.html>здесь</a>

In [ ]:
# ваш код здесь

### 2. Предобработка данных
В данных имеются проблемы. Разделите набор признаков на два поднабора: по времени посещения и по сайтам, которые посещали пользователи.
1. Заполните пропущенные номера сайтов;
2. Проверьте типы данных у атрибутов обеих таблиц. Все временные признаки приведите к типу `datetime` с помощью метода `pd.to_datetime`.

Обратите внимание, что пропуски нужно `заполнять осторожно и обдуманно`, потому что вы можете внести лишние зависимости, которые впоследствии будут мешать корректно идентифицировать аномальных пользователей.

`*опишите здесь, на основе каких рассуждений принято решение о выбранном способе восстановлении данных*`

In [ ]:
# ваш код здесь

### 3. Кластеризация данных 
Поскольку в полученных данных нет меток, определяющих явно пользователя как обычного/аномального, нужно сформировать представление о данных. Необходимо произвести кластеризацию и посмотреть, какие метки и каким пользователям выдаёт алгоритм. Для этого:
1. Обучите* на предобработанных данных алгоритм кластеризации `k-means` и получите метки кластеров. Известно ли в этом кейсе, сколько кластеров требуется для решения задачи? `Прокомментируйте`.

*Перед кластеризацией временные признаки переведите в число и разделите на $10^9$, а затем прошкалируйте. Но при этом временные метки не удаляйте, а отложите в сторону для дальнейших действий.

In [ ]:
# ваш код здесь

2. Обучите на предобработанных данных алгоритм кластеризации по плотности DBSCAN. Получите метки классов и сохраните в отдельный столбец данных.

In [ ]:
# ваш код здесь

3. Пока достаточно трудно проанализировать результаты двух кластеризаций. У нас есть временные метки, которые могут помочь проанализировать результаты кластеризации. Воспользуйтесь методом конструирования признаков и добавьте в набор следующие фичи:
   * час, когда пользователь зашёл на сайт;
   * день недели, когда пользователь зашёл на сайт;
   * бинарный признак, указывающий, что день был выходным днём;
   * посчитать длительность сессии (разница времени между `site10` и `site1`);
   * признак, сообщающий, первая или вторая половина часа, когда пользователь зашёл на сайт;
   * придумать свои дополнительные признаки.

In [ ]:
# ваш код здесь

4. Придумайте, как с помощью сконструированных признаков можно визуализировать разницу между кластерами. 

In [ ]:
# ваш код здесь

5. Обучите алгоритмы кластеризации, но уже с добавлением полученных признаков. Снова постройте визуализации из пункта 4. `Внесли ли новые фичи изменения в результат кластеризации?`

## 4. Конструирование признаков (feature engineering)
Полученных признаков мало, можно ещё добавить фичей. Например:
1. попробуйте создать такой признак, который будет представлять собой число вида `ГГГГММ` от той даты, когда проходила сессия. Например, 201407 - 2014 год и 7 месяц. Таким образом, мы будем учитывать помесячный линейный тренд за весь период предоставленных данных;
2. попробуйте намайнить признаки на основе словаря с сайтами: сформируйте `"мешок" сайтов`*. То есть необходимо создать новую матрицу, в которой строкам будут соответствовать сессии из 10 сайтов, а столбцам – индексы сайтов. На пересечении строки и столбца будет стоять число – количество раз, которое встретился сайт в сессии номер N. 

*Не используйте для второго задания `pandas` и `numpy`! Полученная матрица просто не влезет в память вашего компьютера. Вместо этого используйте класс <a href=https://scikit-learn.org/1.5/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html>CountVectorizer</a> для создания матрицы.

In [ ]:
# ваш код здесь

In [4]:
# и можно ещё здесь

In [ ]:
# подсказка
# CountVectorizer принимает на вход множество текстов,
# поэтому вам необходимо последовательности индексов сайтов для одной сессии
# склеить в строки
# код ниже поможет
def row_to_text(row):
    return ' '.join(map(str, row))
# функцию необходимо применить к каждой строке таблицы с последовательностями индексов

## Поиск аномальных пользователей

На этом этапе у нас должны быть две матрицы: исходные признаки+новые фичи и матрица от CountVectorizer'a. 
1. Склейте две матрицы по вертикали;
2. Произведите кластеризацию методами `k-means` и `dbscan` на обобщённых данных. Сохраните результаты кластеризации в отдельный датафрейм, в котором метки классов расставлены в соответствии с наблюдениями в исходных данных;

In [ ]:
# ваш код здесь

3. Полученный набор данных многомерен. (`Сколько столбцов в полученной матрице?`) Однако хотелось бы визуализировать множество пользователей и попробовать разделить их на чёткие кластеры визуально. Посмотрите метод для понижения размерности, который называется методом стохастически вложенных соседей (<a href=https://nuancesprog.ru/p/14951/>t-SNE</a>) и примените его к вашим данным. Визуализируйте результат. `Видны ли чёткие границы кластеров?` 

In [ ]:
# ваш код здесь

4. Попробуйте использовать модельные методы. Возьмите для начала метод опорных векторов с одним классом `One-Class SVM`. Установите параметр `kernel='linear` и сгенерируйте метки классов. Это позволит модели предполагать, что данные линейно разделимы. Поможет ли это предположение модели в идентификации аномального пользователя? Затем установите параметр `kernel='poly'`, что позволит создать модель, учитывающую нелинейные зависимости. Сгенерируйте метки классов. Пользуйтесь документацией библиотеки `scikit-learn`: [OneClassSVM - документация](https://scikit-learn.org/dev/modules/generated/sklearn.svm.OneClassSVM.html)

In [ ]:
# ваш код здесь

5. После этого используйте более сложную модель изолирующего леса ([`Isolation Forest`](https://scikit-learn.ru/stable/modules/generated/sklearn.ensemble.IsolationForest.html#sklearn.ensemble.IsolationForest)). По своей работе он модель похожа на случайный лес, однако при этом стремится с помощью неглубоких деревьев отделить самые аномальные наблюдения от всей выборки. Подберите гиперпараметры модели вручную, сгенерируйте метки и сохраните их.

In [ ]:
# ваш код здесь

6. Проанализируйте полученные результаты работы модели. Используя визуализации в данных и допустимо ожидаемый процент аномалий, попытайтесь определить, какая модель справилась лучше. Сделайте выводы.

In [ ]:
# ваш код здесь

7. Сохраните метки лучшей модели в файл csv (только колонку с ответами модели!) и отправьте вместе с этим отчётом мне в телегу.